### Churn @ Robinhood
#### Load Libraries and Data

In [1]:
from numba import cuda

try:
    print("Available GPUs:", cuda.gpus)
except cuda.CudaSupportError as e:
    print("CUDA Error:", e)

Available GPUs: <Managed Device 0>


In [2]:
import cudf as cf
equity_df_raw = cf.read_csv('./data/equity_value_data.csv')
features_df_raw = cf.read_csv('./data/features_data.csv')

equity_df = equity_df_raw.copy()
features_df = features_df_raw.copy()

In [3]:
equity_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1119158 entries, 0 to 1119157
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   timestamp     1119158 non-null  object
 1   close_equity  1119158 non-null  float64
 2   user_id       1119158 non-null  object
dtypes: float64(1), object(2)
memory usage: 72.6+ MB


In [4]:
features_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5584 entries, 0 to 5583
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   risk_tolerance                5584 non-null   object
 1   investment_experience         5584 non-null   object
 2   liquidity_needs               5584 non-null   object
 3   platform                      5584 non-null   object
 4   time_spent                    5584 non-null   float64
 5   instrument_type_first_traded  5584 non-null   object
 6   first_deposit_amount          5584 non-null   float64
 7   time_horizon                  5584 non-null   object
 8   user_id                       5584 non-null   object
dtypes: float64(2), object(7)
memory usage: 896.8+ KB


#### a). What percentage of users have churned in the data?
A user is *churned* when their equity falls below 10 usd for 28 consecutive calendar days or longer having perviously been at least 10 usd

**NOTE** Since no equities falls under 10 usd, threshold is set to a variable usd instead

In [5]:
# Set threshold
thresh = 12

# Step 1: Ensure the timestamp column is datetime and sort the data
equity_df['timestamp'] = cf.to_datetime(equity_df['timestamp'])
equity_df = equity_df.sort_values(['user_id', 'timestamp'])

# Step 2: Flag close_equity below threshold
equity_df[f'below_{thresh}'] = (equity_df['close_equity'] < thresh).astype(int)

# Step 3: Compute rolling 28-day windows for each user
equity_df[f'below_{thresh}_28d'] = equity_df.groupby('user_id')[f'below_{thresh}'].rolling(window=28, min_periods=28).sum().reset_index(0, drop=True)

# Step 4: Identify churn (continuous 28 days below $11)
equity_df['churn'] = (equity_df[f'below_{thresh}_28d'] == 28).astype(int).copy()

# Step 5: Check if the user ever had close_equity >= 11
# Group by 'user_id' to find the max close_equity
user_max_equity = equity_df.groupby('user_id')['close_equity'].max().reset_index()
user_max_equity.rename(columns={"close_equity": "max_equity"}, inplace=True)

# Merge back to associate max_equity with each user_id in the main DataFrame
equity_df = equity_df.merge(user_max_equity, on="user_id", how="left")

# Add a flag for users who had close_equity >= 11 at some point
equity_df[f'above_{thresh}_before'] = (equity_df['max_equity'] >= thresh).astype(int)

# Step 6: Filter churned users
churned_users = equity_df.loc[
    (equity_df['churn'] == 1) & (equity_df[f'above_{thresh}_before'] == 1),
    'user_id'
].unique()

# Step 7: Calculate the churn percentage
total_users = equity_df['user_id'].nunique()
churn_percentage = (len(churned_users) / total_users) * 100

print(f"Churned Percentage: {churn_percentage:.2f}%\n")
print(f"Churned Users: {churned_users}\n")
print(f"Total Users: {total_users}\n")

Churned Percentage: 1.58%

Churned Users: 0     0503b35d3715f13fec0a6b7319a32a4f
1     030f4ca7bb7bf14fe5a442b9bffc19e8
2     08c81465c35d7b5208f4c14bd5bcc3bb
3     0c29d08eaf4a53f9c33218de00371311
4     0ee1c5d34c5d6bc0af382cc51eabea5c
                    ...               
83    f0b60bf9e2ceba027dc1f67d0152f4fa
84    f1c5f7bb83a2e6b0afe7d718cde09e1d
85    f55741ace6b614e9017a6b6e19367a7f
86    f9e3f4295ec8017a75ffed3184bc9ce6
87    fc3e52dfa44438cd91838b1f48b7e57a
Name: user_id, Length: 88, dtype: object

Total Users: 5584



Find total days on platform for each user_id

In [6]:
equity_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1119158 entries, 0 to 1119157
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype
---  ------           --------------    -----
 0   timestamp        1119158 non-null  datetime64[ns]
 1   close_equity     1119158 non-null  float64
 2   user_id          1119158 non-null  object
 3   below_12         1119158 non-null  int64
 4   below_12_28d     971628 non-null   int64
 5   churn            971628 non-null   int64
 6   max_equity       1119158 non-null  float64
 7   above_12_before  1119158 non-null  int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 98.5+ MB


In [7]:
# Convert cudf DataFrame to pandas for grouping, if necessary
equity_df = equity_df.to_pandas()

# Group by `user_id` and calculate the total days on platform
total_days = equity_df.groupby('user_id')['timestamp'].agg(lambda x: (x.max() - x.min()).days + 1).reset_index()

# Rename the columns for clarity
total_days.columns = ['user_id', 'total_days_on_platform']

# Display the result
print(total_days)

                               user_id  total_days_on_platform
0     0012db34aa7b083f5714e7831195e54d                     365
1     001d6c77dbdb3213cead7673f250bfdc                     365
2     002e4653171ddc61c3cd30603cd7bd3e                     184
3     00384fa9be6fdca1b786bae70d78f88f                     177
4     0042aac295a0d4df88f4b83012778bd4                     365
...                                ...                     ...
5579  ff9ee08791e20724a86995ab2bc72578                     365
5580  ffa12d2f97e310910291f9b26fb2318d                     365
5581  ffae713096867a32e74f633060667153                     324
5582  ffbda9a14e07718e2b21fb03896d21c5                     185
5583  ffc1e622f3a0b2666f09a6dcb7f27918                     299

[5584 rows x 2 columns]


b). Build a classifier given a user with their features assigns a churn probability for every user and predicts which users will churn

In [8]:
# Create the 'churned' column
features_df['churned'] = features_df['user_id'].isin(churned_users).astype('int32')
features_df['churned'].value_counts()

churned
0    5496
1      88
Name: count, dtype: int64

In [9]:
features_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5584 entries, 0 to 5583
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   risk_tolerance                5584 non-null   object
 1   investment_experience         5584 non-null   object
 2   liquidity_needs               5584 non-null   object
 3   platform                      5584 non-null   object
 4   time_spent                    5584 non-null   float64
 5   instrument_type_first_traded  5584 non-null   object
 6   first_deposit_amount          5584 non-null   float64
 7   time_horizon                  5584 non-null   object
 8   user_id                       5584 non-null   object
 9   churned                       5584 non-null   int32
dtypes: float64(2), int32(1), object(7)
memory usage: 918.7+ KB


In [10]:
features_df.describe().T

,count,mean,std,min,25%,50%,75%,max
time_spent,5584.0,34.509706,155.080551,0.0,2.848908,13.474708,33.823829,8788.32945
first_deposit_amount,5584.0,633.566805,2118.323263,0.0,50.000000,100.000000,500.000000,50000.00000
churned,5584.0,0.015759,0.124554,0.0,0.000000,0.000000,0.000000,1.00000


In [11]:
features_df['risk_tolerance'].value_counts()

risk_tolerance
high_risk_tolerance    3566
med_risk_tolerance     1779
low_risk_tolerance      239
Name: count, dtype: int64

In [12]:
features_df['investment_experience'].value_counts()

investment_experience
limited_investment_exp      2578
no_investment_exp           1796
good_investment_exp         1134
extensive_investment_exp      76
Name: count, dtype: int64

In [13]:
# change features of investment experience to 3
features_df['investment_experience'] = features_df['investment_experience'].replace({
    'extensive_investment_exp': 'good_investment_exp'
})
features_df['investment_experience'].value_counts()

investment_experience
limited_investment_exp    2578
no_investment_exp         1796
good_investment_exp       1210
Name: count, dtype: int64

In [14]:
features_df['liquidity_needs'].value_counts()

liquidity_needs
very_important_liq_need        4217
somewhat_important_liq_need    1109
not_important_liq_need          258
Name: count, dtype: int64

In [15]:
features_df['platform'].value_counts()

platform
iOS        3550
Android    1529
both        505
Name: count, dtype: int64

In [16]:
features_df['instrument_type_first_traded'].value_counts()

instrument_type_first_traded
stock       4827
etp          383
adr          197
mlp           55
reit          55
cef           20
wrt           16
0             13
rlt            9
lp             8
tracking       1
Name: count, dtype: int64

In [17]:
# Use vectorized operations to assign 'non_stock' to all values not equal to 'stock'
features_df['instrument_type_first_traded'] = (
    features_df['instrument_type_first_traded']
    .where(features_df['instrument_type_first_traded'] == 'stock', 'non_stock')
)

# Check the updated value counts
print(features_df['instrument_type_first_traded'].value_counts())

instrument_type_first_traded
stock        4827
non_stock     757
Name: count, dtype: int64


In [18]:
features_df['time_horizon'].value_counts()

time_horizon
short_time_horizon    2833
long_time_horizon     1833
med_time_horizon       918
Name: count, dtype: int64

In [19]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

features_df = features_df.to_pandas()

# Merge total_days into features_df on 'user_id'
features_df = features_df.merge(total_days, on='user_id', how='left')

# Drop `user_id` and `churned` columns for training
features_df = features_df.drop(columns=['user_id'])

# Identify categorical and numerical features
categorical_features = ['risk_tolerance', 'investment_experience', 'liquidity_needs', 'platform', 
                        'instrument_type_first_traded', 'time_horizon']
numerical_features = ['time_spent', 'first_deposit_amount', 'total_days']

# Define preprocessing for numerical and categorical features
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore', min_frequency=0.01)

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define anomaly detection models and parameter grids
param_grids = {
    'IsolationForest': {
        'model__contamination': [0.005, 0.01, 0.02, 0.05],
        'model__max_samples': [128, 256, 'auto'],
        'model__max_features': [0.25, 0.5, 0.75, 1.0]
    },
    'LocalOutlierFactor': {
        'model__n_neighbors': [10, 20, 35, 50],
        'model__leaf_size': [10, 25, 30, 50],
        'model__contamination': [0.005, 0.01, 0.02, 0.05]
    },
    'OneClassSVM': {
        'model__nu': [0.005, 0.01, 0.05, 0.1],
        'model__gamma': ['scale', 'auto'],
        'model__kernel': ['rbf', 'poly', 'linear']
    }
}

true_labels = features_df['churned']  # Ground truth labels for evaluation

for name, param_grid in param_grids.items():
    if name == 'IsolationForest':
        model = IsolationForest(random_state=42)
    elif name == 'LocalOutlierFactor':
        model = LocalOutlierFactor(novelty=True)
    elif name == 'OneClassSVM':
        model = OneClassSVM()

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        scoring='f1_macro',  # Adjust scoring based on your metric of interest
        cv=3
    )

    grid_search.fit(features_df.drop(columns=['churned']), true_labels)
    best_pipeline = grid_search.best_estimator_

    # Predict using the best pipeline
    predictions = best_pipeline.predict(features_df.drop(columns=['churned']))
    binary_predictions = (predictions == 1).astype(int)

    # Print classification report
    print(f"\n{name} Best Parameters: {grid_search.best_params_}\n")
    print(f"{name} Classification Report:\n")
    print(classification_report(true_labels, binary_predictions))

    # Print feature importances if available
    if hasattr(best_pipeline.named_steps['model'], 'feature_importances_'):
        feature_importances = best_pipeline.named_steps['model'].feature_importances_
        feature_names = numerical_features + list(best_pipeline.named_steps['preprocessor']
                                                  .transformers_[1][1]
                                                  .get_feature_names_out(categorical_features))
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importances
        }).sort_values(by='Importance', ascending=False)
        print(f"\n{name} Feature Importances:\n")
        print(importance_df)


ValueError: 
All the 144 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
144 fits failed with the following error:
Traceback (most recent call last):
  File "/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'total_days'

The above exceptio